# 📚 RAG Systémy s Hugging Face

**Autor:** Praut s.r.o. - AI Integration & Business Automation

RAG (Retrieval-Augmented Generation) kombinuje vyhledávání v dokumentech s generováním odpovědí pomocí LLM.

V tomto notebooku se naučíme:
- Základy RAG architektury
- Indexování dokumentů s vektorovým vyhledáváním
- Chunking strategie pro dlouhé dokumenty
- Integrace s lokálními LLM modely
- Produkční RAG pipeline s evaluací

## Architektura RAG

```
Dotaz → Embedding → Vektorové vyhledávání → Relevantní chunky → LLM → Odpověď
                              ↑
                    Vektorová databáze
                    (indexované dokumenty)
```

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers sentence-transformers faiss-cpu langchain langchain-community chromadb pypdf

In [ ]:
import torch
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
import os
import json
from dataclasses import dataclass
import re

# Hugging Face
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer

# Vektorové vyhledávání
import faiss

# Kontrola GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používám zařízení: {device}")

## 1. Document Chunking - Rozdělení Dokumentů

Správné rozdělení dokumentů je klíčové pro kvalitu RAG systému.

In [ ]:
@dataclass
class DocumentChunk:
    """Reprezentace jednoho chunku dokumentu."""
    text: str
    metadata: Dict[str, Any]
    chunk_id: int
    source: str


class DocumentChunker:
    """
    Rozdělí dokumenty na chunky pro indexování.
    Podporuje různé strategie chunkingu.
    """
    
    def __init__(
        self,
        chunk_size: int = 500,
        chunk_overlap: int = 50,
        separator: str = "\n"
    ):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separator = separator
    
    def chunk_by_size(self, text: str, source: str = "unknown") -> List[DocumentChunk]:
        """Rozdělí text na chunky podle velikosti s překryvem."""
        chunks = []
        start = 0
        chunk_id = 0
        
        while start < len(text):
            # Najdeme konec chunku
            end = start + self.chunk_size
            
            # Pokud nejsme na konci, zkusíme ukončit na hranici věty
            if end < len(text):
                # Hledáme tečku, vykřičník nebo otazník
                last_sentence_end = max(
                    text.rfind(".", start, end),
                    text.rfind("!", start, end),
                    text.rfind("?", start, end)
                )
                if last_sentence_end > start:
                    end = last_sentence_end + 1
            
            chunk_text = text[start:end].strip()
            
            if chunk_text:
                chunks.append(DocumentChunk(
                    text=chunk_text,
                    metadata={
                        "start_char": start,
                        "end_char": end,
                        "chunk_size": len(chunk_text)
                    },
                    chunk_id=chunk_id,
                    source=source
                ))
                chunk_id += 1
            
            # Posun s překryvem
            start = end - self.chunk_overlap
        
        return chunks
    
    def chunk_by_paragraphs(self, text: str, source: str = "unknown") -> List[DocumentChunk]:
        """Rozdělí text na chunky podle odstavců."""
        paragraphs = text.split("\n\n")
        chunks = []
        current_chunk = ""
        chunk_id = 0
        
        for para in paragraphs:
            para = para.strip()
            if not para:
                continue
            
            # Pokud by chunk byl příliš velký, uložíme a začneme nový
            if len(current_chunk) + len(para) > self.chunk_size and current_chunk:
                chunks.append(DocumentChunk(
                    text=current_chunk.strip(),
                    metadata={"type": "paragraph"},
                    chunk_id=chunk_id,
                    source=source
                ))
                chunk_id += 1
                current_chunk = ""
            
            current_chunk += para + "\n\n"
        
        # Poslední chunk
        if current_chunk.strip():
            chunks.append(DocumentChunk(
                text=current_chunk.strip(),
                metadata={"type": "paragraph"},
                chunk_id=chunk_id,
                source=source
            ))
        
        return chunks
    
    def chunk_by_sentences(self, text: str, source: str = "unknown", sentences_per_chunk: int = 5) -> List[DocumentChunk]:
        """Rozdělí text na chunky podle vět."""
        # Jednoduchý sentence splitter
        sentences = re.split(r'(?<=[.!?])\s+', text)
        chunks = []
        chunk_id = 0
        
        for i in range(0, len(sentences), sentences_per_chunk):
            chunk_sentences = sentences[i:i + sentences_per_chunk]
            chunk_text = " ".join(chunk_sentences)
            
            if chunk_text.strip():
                chunks.append(DocumentChunk(
                    text=chunk_text.strip(),
                    metadata={
                        "type": "sentences",
                        "sentence_count": len(chunk_sentences)
                    },
                    chunk_id=chunk_id,
                    source=source
                ))
                chunk_id += 1
        
        return chunks

In [ ]:
# Ukázkový dokument
sample_document = """
Umělá inteligence v podnikání

Umělá inteligence (AI) transformuje způsob, jakým firmy fungují. Od automatizace rutinních úkolů po pokročilou analýzu dat, AI nabízí širokou škálu možností pro zvýšení efektivity a konkurenceschopnosti.

Jednou z nejčastějších aplikací AI je zákaznický servis. Chatboty poháněné AI dokáží odpovídat na běžné dotazy zákazníků 24/7, což snižuje náklady na podporu a zlepšuje zákaznickou zkušenost. Moderní chatboty využívají zpracování přirozeného jazyka (NLP) k porozumění kontextu a poskytování relevantních odpovědí.

Další významnou oblastí je prediktivní analytika. AI modely dokáží analyzovat historická data a předpovídat budoucí trendy, což pomáhá firmám lépe plánovat zásoby, optimalizovat ceny a identifikovat potenciální problémy dříve, než nastanou.

Automatizace procesů pomocí AI zahrnuje robotickou automatizaci procesů (RPA), která dokáže převzít opakující se administrativní úkoly. To umožňuje zaměstnancům soustředit se na kreativnější a strategičtější činnosti.

Implementace AI vyžaduje pečlivé plánování. Firmy musí zvážit kvalitu dat, technickou infrastrukturu, školení zaměstnanců a etické aspekty využívání AI. Úspěšná implementace AI může vést k významným úsporám nákladů a zvýšení produktivity.
"""

# Test chunkingu
chunker = DocumentChunker(chunk_size=400, chunk_overlap=50)
chunks = chunker.chunk_by_size(sample_document, source="ai_business.txt")

print(f"Počet chunků: {len(chunks)}")
print("="*50)

for chunk in chunks:
    print(f"\nChunk {chunk.chunk_id}:")
    print(f"Délka: {len(chunk.text)} znaků")
    print(f"Text: {chunk.text[:150]}...")

## 2. Vector Store - Vektorová Databáze

Indexujeme chunky pomocí embeddings a FAISS.

In [ ]:
class VectorStore:
    """
    Vektorová databáze pro RAG systém.
    Používá Sentence Transformers pro embeddings a FAISS pro vyhledávání.
    """
    
    def __init__(
        self,
        embedding_model: str = "all-MiniLM-L6-v2",
        index_type: str = "flat"  # flat, ivf, hnsw
    ):
        print(f"Načítám embedding model: {embedding_model}")
        self.encoder = SentenceTransformer(embedding_model)
        self.embedding_dim = self.encoder.get_sentence_embedding_dimension()
        self.index_type = index_type
        
        # Inicializace FAISS indexu
        self.index = None
        self.chunks: List[DocumentChunk] = []
        
        print(f"Embedding dimenze: {self.embedding_dim}")
    
    def _create_index(self, n_vectors: int):
        """Vytvoří FAISS index podle typu."""
        if self.index_type == "flat":
            # Přesné vyhledávání - nejlepší pro malé datasety
            self.index = faiss.IndexFlatIP(self.embedding_dim)
        elif self.index_type == "ivf":
            # Approximate search - rychlejší pro velké datasety
            n_clusters = min(int(np.sqrt(n_vectors)), 100)
            quantizer = faiss.IndexFlatIP(self.embedding_dim)
            self.index = faiss.IndexIVFFlat(quantizer, self.embedding_dim, n_clusters)
        elif self.index_type == "hnsw":
            # HNSW - rychlý approximate search
            self.index = faiss.IndexHNSWFlat(self.embedding_dim, 32)
        else:
            self.index = faiss.IndexFlatIP(self.embedding_dim)
    
    def add_documents(self, chunks: List[DocumentChunk]):
        """Přidá dokumenty do indexu."""
        if not chunks:
            return
        
        # Vytvoření embeddings
        texts = [chunk.text for chunk in chunks]
        embeddings = self.encoder.encode(
            texts,
            normalize_embeddings=True,
            show_progress_bar=True
        )
        
        # Vytvoření indexu pokud neexistuje
        if self.index is None:
            self._create_index(len(chunks))
            
            # IVF index musí být natrénován
            if self.index_type == "ivf":
                self.index.train(embeddings.astype(np.float32))
        
        # Přidání do indexu
        self.index.add(embeddings.astype(np.float32))
        self.chunks.extend(chunks)
        
        print(f"Přidáno {len(chunks)} chunků. Celkem: {len(self.chunks)}")
    
    def search(
        self,
        query: str,
        top_k: int = 5,
        threshold: float = 0.0
    ) -> List[Tuple[DocumentChunk, float]]:
        """Vyhledá nejrelevantnější chunky pro dotaz."""
        if self.index is None or len(self.chunks) == 0:
            return []
        
        # Embedding dotazu
        query_embedding = self.encoder.encode(
            [query],
            normalize_embeddings=True
        ).astype(np.float32)
        
        # Vyhledávání
        scores, indices = self.index.search(query_embedding, min(top_k, len(self.chunks)))
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx >= 0 and score >= threshold:
                results.append((self.chunks[idx], float(score)))
        
        return results
    
    def save(self, path: str):
        """Uloží index a metadata."""
        os.makedirs(path, exist_ok=True)
        
        # Uložení FAISS indexu
        faiss.write_index(self.index, f"{path}/index.faiss")
        
        # Uložení chunků
        chunks_data = [
            {
                "text": c.text,
                "metadata": c.metadata,
                "chunk_id": c.chunk_id,
                "source": c.source
            }
            for c in self.chunks
        ]
        with open(f"{path}/chunks.json", "w", encoding="utf-8") as f:
            json.dump(chunks_data, f, ensure_ascii=False, indent=2)
        
        print(f"Index uložen do: {path}")
    
    def load(self, path: str):
        """Načte index a metadata."""
        # Načtení FAISS indexu
        self.index = faiss.read_index(f"{path}/index.faiss")
        
        # Načtení chunků
        with open(f"{path}/chunks.json", "r", encoding="utf-8") as f:
            chunks_data = json.load(f)
        
        self.chunks = [
            DocumentChunk(
                text=c["text"],
                metadata=c["metadata"],
                chunk_id=c["chunk_id"],
                source=c["source"]
            )
            for c in chunks_data
        ]
        
        print(f"Načteno {len(self.chunks)} chunků z: {path}")

In [ ]:
# Vytvoření a naplnění vektorové databáze
vector_store = VectorStore(embedding_model="all-MiniLM-L6-v2")

# Přidání více dokumentů
documents = [
    {
        "source": "ai_business.txt",
        "text": sample_document
    },
    {
        "source": "customer_service.txt",
        "text": """
        Zákaznický servis a podpora
        
        Kvalitní zákaznický servis je základem úspěšného podnikání. Spokojený zákazník se vrací a doporučuje firmu dalším.
        
        Základní principy kvalitního servisu zahrnují rychlou reakci na dotazy, empatický přístup a efektivní řešení problémů.
        Moderní technologie jako CRM systémy pomáhají sledovat historii komunikace a personalizovat služby.
        
        Chatboty dokáží odbavit rutinní dotazy automaticky, ale složitější případy vyžadují lidský přístup.
        Důležité je nastavit správnou eskalaci - kdy předat případ živému operátorovi.
        
        Měření spokojenosti zákazníků pomocí NPS (Net Promoter Score) a CSAT pomáhá identifikovat oblasti pro zlepšení.
        """
    },
    {
        "source": "ecommerce.txt",
        "text": """
        E-commerce a online prodej
        
        Online obchody zažívají boom. Pro úspěch v e-commerce je klíčová uživatelská zkušenost, rychlé načítání stránek a důvěryhodnost.
        
        Optimalizace konverzního poměru zahrnuje A/B testování, personalizaci nabídek a zjednodušení checkout procesu.
        Každý krok navíc v nákupním procesu snižuje konverzi o 10-15%.
        
        Logistika a fulfillment jsou kritické. Zákazníci očekávají rychlé doručení a snadné vrácení zboží.
        Same-day delivery se stává standardem ve větších městech.
        
        Platební brány musí podporovat různé metody - karty, Apple Pay, Google Pay, bankovní převody.
        Bezpečnost plateb je prioritou - PCI DSS compliance je nutností.
        """
    }
]

# Chunk a indexace všech dokumentů
chunker = DocumentChunker(chunk_size=300, chunk_overlap=30)
all_chunks = []

for doc in documents:
    doc_chunks = chunker.chunk_by_size(doc["text"], source=doc["source"])
    all_chunks.extend(doc_chunks)

vector_store.add_documents(all_chunks)

In [ ]:
# Test vyhledávání
test_queries = [
    "Jak může AI pomoci se zákaznickým servisem?",
    "Co je důležité pro e-shop?",
    "Jak měřit spokojenost zákazníků?",
    "Jaké jsou výhody chatbotů?"
]

for query in test_queries:
    print(f"\nDotaz: {query}")
    print("-" * 50)
    
    results = vector_store.search(query, top_k=2)
    for chunk, score in results:
        print(f"  [{score:.3f}] {chunk.source}: {chunk.text[:100]}...")

## 3. RAG Pipeline - Kompletní Systém

Spojíme vyhledávání s generováním odpovědí pomocí LLM.

In [ ]:
class RAGPipeline:
    """
    Kompletní RAG pipeline pro question answering.
    """
    
    def __init__(
        self,
        vector_store: VectorStore,
        llm_model: str = "google/flan-t5-base",
        top_k: int = 3,
        max_context_length: int = 1500
    ):
        self.vector_store = vector_store
        self.top_k = top_k
        self.max_context_length = max_context_length
        
        # Načtení LLM
        print(f"Načítám LLM: {llm_model}")
        self.generator = pipeline(
            "text2text-generation",
            model=llm_model,
            device=0 if torch.cuda.is_available() else -1,
            max_length=512
        )
        print("LLM načten.")
    
    def _build_context(self, chunks: List[Tuple[DocumentChunk, float]]) -> str:
        """Sestaví kontext z nalezených chunků."""
        context_parts = []
        total_length = 0
        
        for chunk, score in chunks:
            if total_length + len(chunk.text) > self.max_context_length:
                break
            context_parts.append(chunk.text)
            total_length += len(chunk.text)
        
        return "\n\n".join(context_parts)
    
    def _build_prompt(self, query: str, context: str) -> str:
        """Vytvoří prompt pro LLM."""
        prompt = f"""Na základě následujícího kontextu odpověz na otázku. 
Pokud odpověď není v kontextu, řekni že nevíš.

Kontext:
{context}

Otázka: {query}

Odpověď:"""
        return prompt
    
    def query(
        self,
        question: str,
        return_sources: bool = True
    ) -> Dict[str, Any]:
        """Zpracuje dotaz a vrátí odpověď."""
        # 1. Vyhledání relevantních dokumentů
        retrieved = self.vector_store.search(question, top_k=self.top_k)
        
        if not retrieved:
            return {
                "answer": "Nenašel jsem žádné relevantní informace.",
                "sources": [],
                "context": ""
            }
        
        # 2. Sestavení kontextu
        context = self._build_context(retrieved)
        
        # 3. Generování odpovědi
        prompt = self._build_prompt(question, context)
        response = self.generator(prompt)[0]["generated_text"]
        
        # 4. Příprava výsledku
        result = {
            "answer": response.strip(),
            "context": context
        }
        
        if return_sources:
            result["sources"] = [
                {
                    "source": chunk.source,
                    "score": score,
                    "text": chunk.text[:200] + "..."
                }
                for chunk, score in retrieved
            ]
        
        return result
    
    def batch_query(self, questions: List[str]) -> List[Dict[str, Any]]:
        """Zpracuje více dotazů najednou."""
        return [self.query(q) for q in questions]

In [ ]:
# Vytvoření RAG pipeline
rag = RAGPipeline(
    vector_store=vector_store,
    llm_model="google/flan-t5-base",
    top_k=3
)

In [ ]:
# Test RAG pipeline
questions = [
    "Jak může AI pomoci v zákaznickém servisu?",
    "Co je důležité pro úspěch e-shopu?",
    "Jak měřit spokojenost zákazníků?"
]

for question in questions:
    print("\n" + "="*60)
    print(f"OTÁZKA: {question}")
    print("="*60)
    
    result = rag.query(question)
    
    print(f"\nODPOVĚĎ: {result['answer']}")
    print(f"\nZDROJE:")
    for source in result["sources"]:
        print(f"  - {source['source']} (score: {source['score']:.3f})")

## 4. Pokročilý RAG s Re-ranking

Přidáme cross-encoder pro lepší hodnocení relevance.

In [ ]:
from sentence_transformers import CrossEncoder

class AdvancedRAGPipeline:
    """
    Pokročilý RAG s re-rankingem a hybrid search.
    """
    
    def __init__(
        self,
        vector_store: VectorStore,
        llm_model: str = "google/flan-t5-base",
        reranker_model: str = "cross-encoder/ms-marco-MiniLM-L-6-v2",
        initial_k: int = 10,
        final_k: int = 3
    ):
        self.vector_store = vector_store
        self.initial_k = initial_k
        self.final_k = final_k
        
        # LLM
        print(f"Načítám LLM: {llm_model}")
        self.generator = pipeline(
            "text2text-generation",
            model=llm_model,
            device=0 if torch.cuda.is_available() else -1,
            max_length=512
        )
        
        # Re-ranker (Cross-encoder)
        print(f"Načítám re-ranker: {reranker_model}")
        self.reranker = CrossEncoder(reranker_model)
        print("Modely načteny.")
    
    def _rerank(self, query: str, chunks: List[Tuple[DocumentChunk, float]]) -> List[Tuple[DocumentChunk, float]]:
        """Re-rankuje výsledky pomocí cross-encoderu."""
        if not chunks:
            return []
        
        # Příprava párů pro cross-encoder
        pairs = [(query, chunk.text) for chunk, _ in chunks]
        
        # Skóre z cross-encoderu
        scores = self.reranker.predict(pairs)
        
        # Kombinace s původním skóre
        reranked = []
        for (chunk, original_score), new_score in zip(chunks, scores):
            combined_score = 0.3 * original_score + 0.7 * new_score  # Váha pro cross-encoder
            reranked.append((chunk, combined_score))
        
        # Seřazení podle nového skóre
        reranked.sort(key=lambda x: x[1], reverse=True)
        
        return reranked[:self.final_k]
    
    def query(
        self,
        question: str,
        use_reranking: bool = True
    ) -> Dict[str, Any]:
        """Zpracuje dotaz s volitelným re-rankingem."""
        # 1. Inicialní vyhledávání (více výsledků)
        initial_results = self.vector_store.search(question, top_k=self.initial_k)
        
        if not initial_results:
            return {"answer": "Nenašel jsem relevantní informace.", "sources": []}
        
        # 2. Re-ranking (pokud je povolen)
        if use_reranking:
            final_results = self._rerank(question, initial_results)
        else:
            final_results = initial_results[:self.final_k]
        
        # 3. Sestavení kontextu
        context = "\n\n".join([chunk.text for chunk, _ in final_results])
        
        # 4. Generování odpovědi
        prompt = f"""Odpověz na otázku na základě kontextu. Buď stručný a konkrétní.

Kontext:
{context}

Otázka: {question}

Odpověď:"""
        
        response = self.generator(prompt)[0]["generated_text"]
        
        return {
            "answer": response.strip(),
            "sources": [
                {"source": c.source, "score": s, "text": c.text[:150]}
                for c, s in final_results
            ],
            "reranked": use_reranking
        }

In [ ]:
# Test pokročilého RAG
advanced_rag = AdvancedRAGPipeline(
    vector_store=vector_store,
    llm_model="google/flan-t5-base",
    initial_k=10,
    final_k=3
)

question = "Jaké technologie pomáhají v zákaznickém servisu?"

# Bez re-rankingu
result_basic = advanced_rag.query(question, use_reranking=False)
print("BEZ RE-RANKINGU:")
print(f"Odpověď: {result_basic['answer']}")

print("\n" + "="*50)

# S re-rankingem
result_reranked = advanced_rag.query(question, use_reranking=True)
print("S RE-RANKINGEM:")
print(f"Odpověď: {result_reranked['answer']}")

## 5. Produkční RAG Systém

Kompletní systém s cachingem, logováním a evaluací.

In [ ]:
from datetime import datetime
import hashlib
from collections import OrderedDict

class ProductionRAG:
    """
    Produkční RAG systém s pokročilými funkcemi.
    """
    
    def __init__(
        self,
        embedding_model: str = "all-MiniLM-L6-v2",
        llm_model: str = "google/flan-t5-base",
        cache_size: int = 100
    ):
        # Komponenty
        self.chunker = DocumentChunker(chunk_size=400, chunk_overlap=50)
        self.vector_store = VectorStore(embedding_model=embedding_model)
        
        # LLM
        self.generator = pipeline(
            "text2text-generation",
            model=llm_model,
            device=0 if torch.cuda.is_available() else -1,
            max_length=512
        )
        
        # Cache
        self.cache = OrderedDict()
        self.cache_size = cache_size
        
        # Statistiky
        self.stats = {
            "total_queries": 0,
            "cache_hits": 0,
            "avg_retrieval_time": 0,
            "avg_generation_time": 0
        }
        
        # Historie dotazů
        self.query_history = []
    
    def add_document(self, text: str, source: str, metadata: Dict = None):
        """Přidá dokument do systému."""
        chunks = self.chunker.chunk_by_size(text, source=source)
        
        # Přidání metadat
        if metadata:
            for chunk in chunks:
                chunk.metadata.update(metadata)
        
        self.vector_store.add_documents(chunks)
        return len(chunks)
    
    def add_documents_from_folder(self, folder_path: str, extensions: List[str] = [".txt", ".md"]):
        """Načte dokumenty ze složky."""
        total_chunks = 0
        
        for filename in os.listdir(folder_path):
            if any(filename.endswith(ext) for ext in extensions):
                filepath = os.path.join(folder_path, filename)
                with open(filepath, "r", encoding="utf-8") as f:
                    text = f.read()
                
                chunks = self.add_document(text, source=filename)
                total_chunks += chunks
                print(f"  Načteno {filename}: {chunks} chunků")
        
        return total_chunks
    
    def _get_cache_key(self, query: str) -> str:
        """Vytvoří klíč pro cache."""
        return hashlib.md5(query.lower().strip().encode()).hexdigest()
    
    def _update_cache(self, key: str, value: Dict):
        """Aktualizuje cache s LRU strategií."""
        if key in self.cache:
            self.cache.move_to_end(key)
        else:
            if len(self.cache) >= self.cache_size:
                self.cache.popitem(last=False)
            self.cache[key] = value
    
    def query(
        self,
        question: str,
        top_k: int = 3,
        use_cache: bool = True,
        temperature: float = 0.7
    ) -> Dict[str, Any]:
        """Zpracuje dotaz."""
        import time
        start_time = time.time()
        
        self.stats["total_queries"] += 1
        cache_key = self._get_cache_key(question)
        
        # Cache check
        if use_cache and cache_key in self.cache:
            self.stats["cache_hits"] += 1
            result = self.cache[cache_key].copy()
            result["cached"] = True
            return result
        
        # Retrieval
        retrieval_start = time.time()
        retrieved = self.vector_store.search(question, top_k=top_k)
        retrieval_time = time.time() - retrieval_start
        
        if not retrieved:
            return {"answer": "Nenašel jsem relevantní informace.", "sources": []}
        
        # Build context
        context = "\n\n".join([chunk.text for chunk, _ in retrieved])
        
        # Generation
        generation_start = time.time()
        prompt = f"""Odpověz na otázku na základě kontextu. Odpověď by měla být stručná a přesná.

Kontext:
{context}

Otázka: {question}

Odpověď:"""
        
        response = self.generator(
            prompt,
            do_sample=temperature > 0,
            temperature=temperature if temperature > 0 else None
        )[0]["generated_text"]
        generation_time = time.time() - generation_start
        
        # Prepare result
        result = {
            "answer": response.strip(),
            "sources": [
                {"source": c.source, "score": float(s), "chunk_id": c.chunk_id}
                for c, s in retrieved
            ],
            "metadata": {
                "retrieval_time": retrieval_time,
                "generation_time": generation_time,
                "total_time": time.time() - start_time,
                "top_k": top_k
            },
            "cached": False
        }
        
        # Update cache
        if use_cache:
            self._update_cache(cache_key, result)
        
        # Log query
        self.query_history.append({
            "timestamp": datetime.now().isoformat(),
            "question": question,
            "answer": result["answer"],
            "sources_count": len(result["sources"]),
            "time": result["metadata"]["total_time"]
        })
        
        return result
    
    def get_statistics(self) -> Dict[str, Any]:
        """Vrátí statistiky systému."""
        cache_hit_rate = (
            self.stats["cache_hits"] / self.stats["total_queries"]
            if self.stats["total_queries"] > 0 else 0
        )
        
        return {
            "total_queries": self.stats["total_queries"],
            "cache_hit_rate": cache_hit_rate,
            "documents_indexed": len(self.vector_store.chunks),
            "cache_size": len(self.cache),
            "recent_queries": self.query_history[-5:] if self.query_history else []
        }
    
    def evaluate(
        self,
        test_questions: List[str],
        expected_sources: List[str] = None
    ) -> Dict[str, float]:
        """Evaluuje systém na testovacích datech."""
        results = {
            "total_questions": len(test_questions),
            "avg_sources_returned": 0,
            "avg_top_score": 0,
            "source_accuracy": 0 if expected_sources else None
        }
        
        total_sources = 0
        total_top_score = 0
        correct_sources = 0
        
        for i, question in enumerate(test_questions):
            response = self.query(question, use_cache=False)
            
            sources = response.get("sources", [])
            total_sources += len(sources)
            
            if sources:
                total_top_score += sources[0]["score"]
            
            if expected_sources and i < len(expected_sources):
                if any(s["source"] == expected_sources[i] for s in sources):
                    correct_sources += 1
        
        results["avg_sources_returned"] = total_sources / len(test_questions)
        results["avg_top_score"] = total_top_score / len(test_questions)
        
        if expected_sources:
            results["source_accuracy"] = correct_sources / len(test_questions)
        
        return results
    
    def save(self, path: str):
        """Uloží systém."""
        os.makedirs(path, exist_ok=True)
        self.vector_store.save(f"{path}/vector_store")
        
        # Uložení statistik
        with open(f"{path}/stats.json", "w") as f:
            json.dump(self.stats, f)
        
        print(f"RAG systém uložen do: {path}")

In [ ]:
# Vytvoření produkčního RAG systému
prod_rag = ProductionRAG(
    embedding_model="all-MiniLM-L6-v2",
    llm_model="google/flan-t5-base",
    cache_size=50
)

# Přidání dokumentů
for doc in documents:
    prod_rag.add_document(doc["text"], source=doc["source"])

In [ ]:
# Test produkčního RAG
test_questions = [
    "Co je prediktivní analytika?",
    "Jak zlepšit konverzi v e-shopu?",
    "Jaké jsou výhody chatbotů?",
    "Co je NPS?"
]

print("TESTOVÁNÍ PRODUKČNÍHO RAG SYSTÉMU")
print("="*60)

for q in test_questions:
    result = prod_rag.query(q)
    print(f"\nOtázka: {q}")
    print(f"Odpověď: {result['answer']}")
    print(f"Čas: {result['metadata']['total_time']:.3f}s")
    print(f"Cached: {result['cached']}")

# Statistiky
print("\n" + "="*60)
print("STATISTIKY:")
stats = prod_rag.get_statistics()
for key, value in stats.items():
    if key != "recent_queries":
        print(f"  {key}: {value}")

In [ ]:
# Test cachingu - druhý dotaz by měl být z cache
print("TEST CACHINGU:")
print("-" * 40)

question = "Co je prediktivní analytika?"

# První dotaz
result1 = prod_rag.query(question)
print(f"První dotaz - Čas: {result1['metadata']['total_time']:.3f}s, Cached: {result1['cached']}")

# Druhý dotaz (z cache)
result2 = prod_rag.query(question)
print(f"Druhý dotaz - Cached: {result2['cached']}")

# Statistiky
stats = prod_rag.get_statistics()
print(f"\nCache hit rate: {stats['cache_hit_rate']:.2%}")

In [ ]:
# Evaluace systému
eval_questions = [
    "Jak AI pomáhá firmám?",
    "Co je důležité pro e-commerce?",
    "Jak funguje zákaznický servis?",
]

expected = ["ai_business.txt", "ecommerce.txt", "customer_service.txt"]

eval_results = prod_rag.evaluate(eval_questions, expected_sources=expected)

print("EVALUACE SYSTÉMU:")
print("="*40)
for key, value in eval_results.items():
    if value is not None:
        print(f"  {key}: {value:.3f}" if isinstance(value, float) else f"  {key}: {value}")

## Shrnutí

V tomto notebooku jsme implementovali:

1. **Document Chunking** - různé strategie rozdělení dokumentů
2. **Vector Store** - FAISS index pro rychlé vyhledávání
3. **Základní RAG Pipeline** - retrieval + generation
4. **Pokročilý RAG** - s cross-encoder re-rankingem
5. **Produkční RAG** - s cachingem, statistikami a evaluací

### Další vylepšení pro produkci:
- Hybrid search (BM25 + vektory)
- Streaming odpovědí
- Multi-modal RAG (obrázky, tabulky)
- RAG s citacemi
- Fine-tuning embeddings na doménových datech